In [4]:
import os
import numpy as np
import pandas as pd

In [18]:
FOLDER = "out"
new_folder = "to_overleaf"

In [6]:
exps = os.listdir(FOLDER)
exps

['3x_1 - 2x_2_elastic',
 '3x_1 - 2x_2_L1',
 '3x_1 - 2x_2_L2',
 'california_housing_elastic',
 'california_housing_L1',
 'california_housing_L2',
 'diabetes_elastic',
 'diabetes_L1',
 'diabetes_L2',
 'optim_slides_elastic',
 'optim_slides_L1',
 'optim_slides_L2']

In [36]:
import json
import shutil

# Ensure the new folder exists
os.makedirs(new_folder, exist_ok=True)

for exp in exps:
    if not os.path.exists(os.path.join(new_folder, exp)):
        os.makedirs(os.path.join(new_folder, exp), exist_ok=True)
    for file in os.listdir(os.path.join(FOLDER, exp)):
        if file.endswith(".png") and "combined" in file:
            # copy file to new folder
            src_path = os.path.join(FOLDER, exp, file)
            dest_path = os.path.join(new_folder, exp, file)
            shutil.copy(src_path, dest_path)

        with open(os.path.join(FOLDER, exp, "report.json"), "r") as f:
            reports = json.load(f)
        with open(os.path.join(new_folder, exp, "report.json"), "w") as f:
            json.dump(reports, f, indent=4)

In [ ]:
latex_code = r"""\documentclass{article}
\usepackage{graphicx} % Required for inserting images
\usepackage{hyperref} % For URLs and links
\usepackage{amsmath} % For math environments
\usepackage{booktabs} % For better tables
\usepackage{float} % For precise figure placement

\hypersetup{
    colorlinks=true,
    urlcolor=blue
}

\title{Lasso Lambda T}
\author{...}
\date{December 2024}

\begin{document}

\maketitle
"""

for exp in exps:
    exp_path = os.path.join(new_folder, exp)
    report_path = os.path.join(exp_path, "report.json")

    with open(report_path, "r") as f:
        reports = json.load(f)

    # Extract details from the report
    dgp = reports["DGP"]
    reg = reports["REGULARIZATION"]
    n_samples = reports["N_SAMPLES"]
    n_features = reports["N_FEATURES"]

    latex_code += fr"""\subsection*{{Experiment: DGP={dgp}, Regularization={reg}}}
    \begin{{itemize}}
        \item Number of samples: {n_samples}
        \item Number of features: {n_features}
    \end{{itemize}}
    """

    # Add images in a grid (2x2 format)
    imgs = [f for f in os.listdir(exp_path) if f.endswith(".png")]
    if imgs:
        latex_code += r"""\begin{figure}[H]
            \centering
        """
        for i, img in enumerate(imgs):
            img_path = f"{new_folder}/{exp}/{img}".replace("\\", "/")
            latex_code += fr"""
            \includegraphics[width=0.45\textwidth]{{{img_path}}}
            """
            if i % 2 == 1:  # Add vertical space after every two images
                latex_code += r"""
            \vspace{1cm}
            """
        latex_code += r"""
        \end{figure}
        """

    # Process and add L1 and L2 norm tables
    def process_table(df):
        """Process the DataFrame to rename columns and round values."""
        df = df.T  # Transpose the DataFrame
        df = df.rename(columns={"p_val": "P Value", "pearson": "Pearson", 
                                "error": "Error"})  # Rename 'p_val' to 'P Value'
        df = df.round(2)  # Round all numeric values to two decimal places
        df = df.sort_values("R2", ascending=False)  # Sort by R2 in descending order
        return df

    # Process L1 and L2 norm tables
    l1_norm = process_table(pd.DataFrame(reports["relationship_l1_norm"]))
    l2_norm = process_table(pd.DataFrame(reports["relationship_l2_norm"]))

    l1_norm_latex = l1_norm.to_latex(index=True, caption="L1 Norm Relationships", label=f"tab:l1_{exp}")
    l2_norm_latex = l2_norm.to_latex(index=True, caption="L2 Norm Relationships", label=f"tab:l2_{exp}")

    latex_code += fr"""\subsection*{{Relationships}}
    \subsubsection*{{L1 Norm}}
    {l1_norm_latex}
    \subsubsection*{{L2 Norm}}
    {l2_norm_latex}
    """

# Add a Resources section
latex_code += r"""
\section*{Resources}
The codebase for reproducing the results can be found at
\href{https://github.com/HaykTarkhanyan/regularization_lambda_t}{GitHub}.
\end{document}
"""

# save the latex code to a file
with open("report_4.tex", "w") as f:
    f.write(latex_code)

In [35]:
rep_file = r"out\optim_slides_L2\report.json"

with open(rep_file, "r") as f:
    reports = json.load(f)
    
rel_df = pd.DataFrame(reports["relationship_l1_norm"]).T.round(2).sort_values("R2", ascending=False)
rel_df.to_latex()


'\\begin{tabular}{lrrr}\n\\toprule\n & R2 & pearson & p_val \\\\\n\\midrule\nlog & 0.700000 & -0.840000 & 0.000000 \\\\\ncubic & 0.620000 & -0.570000 & 0.000000 \\\\\nsqrt & 0.610000 & -0.780000 & 0.000000 \\\\\nquadratic & 0.500000 & -0.570000 & 0.000000 \\\\\nlinear & 0.330000 & -0.570000 & 0.000000 \\\\\nexp & -2.690000 & -0.810000 & 0.000000 \\\\\nsigmoid & -3.270000 & -0.830000 & 0.000000 \\\\\ntanh & -3.270000 & -0.900000 & 0.000000 \\\\\n1/x & -3.600000 & 1.000000 & 0.000000 \\\\\n1/x^2 & -8.020000 & 0.950000 & 0.000000 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [11]:
import os
import json

def compile_reports_to_markdown(base_path, output_file):
    # Initialize markdown content
    markdown_content = "# Combined Report\n\n"

    # Iterate through all folders
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)

        # Ensure the folder contains a `report.json`
        report_file = os.path.join(folder_path, "report.json")
        if os.path.isdir(folder_path) and os.path.exists(report_file):
            with open(report_file, "r") as f:
                report_data = json.load(f)

            # Start a section for the folder
            markdown_content += f"## Folder: {folder}\n\n"
            markdown_content += f"**Datetime:** {report_data.get('datetime')}\n\n"
            markdown_content += f"**DGP:** {report_data.get('DGP')}\n\n"
            markdown_content += f"**Regularization:** {report_data.get('REGULARIZATION')}\n\n"

            # Include relevant images
            markdown_content += "### Figures\n\n"
            for key, value in report_data.items():
                if isinstance(value, str) and "combined" in value and value.endswith(".html"):
                    image_path = os.path.join(folder, value)
                    image_path = image_path.replace(".html", ".png").replace("figures", "out")
                    markdown_content += f"![{key}]({image_path})\n\n"

            # Include statistical summaries
            markdown_content += "### Relationships\n\n"
            for norm_type in ["relationship_l1_norm", "relationship_l2_norm"]:
                if norm_type in report_data:
                    markdown_content += f"#### {norm_type.replace('_', ' ').title()}\n\n"
                    for relationship, metrics in report_data[norm_type].items():
                        markdown_content += f"- **{relationship.capitalize()}**: "
                        markdown_content += ", ".join(
                            [f"{k}: {v}" for k, v in metrics.items()]
                        ) + "\n"

            markdown_content += "\n---\n\n"

    # Write markdown to file
    with open(output_file, "w") as f:
        f.write(markdown_content)

    print(f"Markdown report compiled successfully to {output_file}")


# Base path to folders and output file
base_path = "out"  # Adjust to your folder structure
output_file = "compiled_report.md"

# Compile reports to markdown
compile_reports_to_markdown(base_path, output_file)


Markdown report compiled successfully to compiled_report.md


In [17]:
import os
import json

def create_markdown_for_folder(folder_path, output_file):
    # Ensure the folder contains a `report.json`
    report_file = os.path.join(folder_path, "report.json")
    if not os.path.exists(report_file):
        print(f"No report.json found in {folder_path}")
        return

    # Load the JSON data
    with open(report_file, "r") as f:
        report_data = json.load(f)

    # Initialize markdown content
    markdown_content = f"# Report for Folder: {os.path.basename(folder_path)}\n\n"

    # Include all keys and values from report.json
    markdown_content += "## Report Details\n\n"
    for key, value in report_data.items():
        if isinstance(value, dict):
            markdown_content += f"**{key.replace('_', ' ').title()}:**\n"
            for sub_key, sub_value in value.items():
                if isinstance(sub_value, dict):
                    markdown_content += f"- **{sub_key.title()}**: " + ", ".join(
                        [f"{k}: {v}" for k, v in sub_value.items()]
                    ) + "\n"
                else:
                    markdown_content += f"- **{sub_key.title()}**: {sub_value}\n"
        else:
            markdown_content += f"**{key.replace('_', ' ').title()}:** {value}\n"

    # Include relevant images
    markdown_content += "\n## Figures\n\n"
    for key, value in report_data.items():
        if isinstance(value, str) and "combined" in value and value.endswith(".html"):
            # Adjust the image path
            image_path = value.replace(".html", ".png").replace("figures", "out")
            markdown_content += f"![{key}]({image_path})\n\n"

    # Include statistical summaries
    markdown_content += "## Relationships\n\n"
    for norm_type in ["relationship_l1_norm", "relationship_l2_norm"]:
        if norm_type in report_data:
            markdown_content += f"### {norm_type.replace('_', ' ').title()}\n\n"
            for relationship, metrics in report_data[norm_type].items():
                metrics_str = ", ".join(
                    [f"{k}: {v}" for k, v in metrics.items()]
                )
                markdown_content += f"- **{relationship.capitalize()}**: {metrics_str}\n"

    # Write markdown to file
    with open(output_file, "w") as f:
        f.write(markdown_content)

    print(f"Markdown report created successfully: {output_file}")


# Folder path and output file
folder_path = "out/3x_1 - 2x_2_elastic"  # Folder containing the report.json
output_file = "folder_report.md"

# Create the markdown report
create_markdown_for_folder(folder_path, output_file)


Markdown report created successfully: folder_report.md
